In [1]:
!npm install dukascopy-node --save


up to date, audited 34 packages in 2s

1 package is looking for funding
  run `npm fund` for details

found 0 vulnerabilities


npm WARN config global `--global`, `--local` are deprecated. Use `--location=global` instead.


In [2]:
import datetime
import pytz

gmt_timezone = pytz.timezone('GMT')
current_date_gmt = datetime.datetime.now(gmt_timezone).date()
current_date_gmt

datetime.date(2023, 5, 9)

In [3]:
from datetime import datetime, timedelta

# Obter a data de 4 dias atrás
four_days_ago = datetime.today() - timedelta(days=5)

# Obter a data atual
today = datetime.today()

# Formatar as datas no formato desejado
from_date = four_days_ago.strftime("%Y-%m-%d")
now = current_date_gmt.strftime("%Y-%m-%d")

In [4]:
!npx dukascopy-node -i usdcad -from "{from_date}" -t h1 -f json --volumes


----------------------------------------------------
----------------------------------------------------
Instrument:     US Dollar vs Canadian Dollar
Timeframe:      h1
From date:      May 4, 2023, 12 AM
To date:        May 9, 2023, 5 PM
Price type:     bid
Volumes:        true
UTC Offset:     0
Include flats:  false
Format:         json
----------------------------------------------------
----------------------------------------------------
âˆš File saved: download\usdcad-h1-bid-2023-05-04-2023-05-09.json (3.4 KB)

Download time: 6.6s



npm WARN config global `--global`, `--local` are deprecated. Use `--location=global` instead.


In [5]:
!npx dukascopy-node -i brentcmdusd -from "{from_date}" -t h1 -f json --volumes


----------------------------------------------------
----------------------------------------------------
Instrument:     US Brent Crude Oil
Timeframe:      h1
From date:      May 4, 2023, 12 AM
To date:        May 9, 2023, 5 PM
Price type:     bid
Volumes:        true
UTC Offset:     0
Include flats:  false
Format:         json
----------------------------------------------------
----------------------------------------------------
âˆš File saved: download\brentcmdusd-h1-bid-2023-05-04-2023-05-09.json (3.25 KB)

Download time: 6.8s



npm WARN config global `--global`, `--local` are deprecated. Use `--location=global` instead.


In [6]:
def add_bollinger_bands(df, period=20):
    # Calcula a média móvel simples
    df['SMA'] = df['Close_usdcad'].rolling(window=period).mean()

    # Calcula o desvio padrão
    df['STD'] = df['Close_usdcad'].rolling(window=period).std()

    # Calcula a banda superior
    df['Upper'] = df['SMA'] + (2 * df['STD'])

    # Calcula a banda inferior
    df['Lower'] = df['SMA'] - (2 * df['STD'])

    # Retorna o dataframe com as bandas de Bollinger adicionadas
    return df


In [7]:
def add_stochastic_indicator(df, period=14):
    # Calcula o preço mínimo dos últimos 14 dias
    low_min = df['Low_usdcad'].rolling(window=period).min()

    # Calcula o preço máximo dos últimos 14 dias
    high_max = df['High_usdcad'].rolling(window=period).max()

    # Calcula a diferença entre o preço de fechamento atual e o preço mínimo do período
    close_diff = df['Close_usdcad'] - low_min

    # Calcula a diferença entre o preço máximo e mínimo do período
    high_diff = high_max - low_min

    # Calcula a porcentagem da diferença entre o preço atual e o preço mínimo em relação à diferença entre o preço máximo e mínimo do período
    stoch = close_diff / high_diff * 100

    # Adiciona o resultado como uma nova coluna ao conjunto de dados
    df['Stochastic'] = stoch

    return df


In [8]:
def add_rsi(df, period=14):
    """
    Adiciona a coluna RSI ao DataFrame fornecido.

    Args:
        df (pd.DataFrame): DataFrame contendo dados OHLC de um ativo.
        period (int): Período de tempo para calcular o RSI.

    Returns:
        pd.DataFrame: DataFrame original com a coluna RSI adicionada.
    """
    # Calcula as diferenças entre o preço de fechamento atual e o preço de fechamento do dia anterior
    diff = df['Close_usdcad'].diff()

    # Separa as diferenças em ganhos e perdas
    gain = diff.where(diff > 0, 0)
    loss = -diff.where(diff < 0, 0)

    # Calcula a média móvel exponencial dos ganhos e perdas
    ema_gain = gain.ewm(alpha=1/period, min_periods=period, adjust=False).mean()
    ema_loss = loss.ewm(alpha=1/period, min_periods=period, adjust=False).mean()

    # Calcula a relação entre a média móvel dos ganhos e perdas, obtendo o RSI
    rs = ema_gain / ema_loss
    rsi = 100 - (100 / (1 + rs))

    # Adiciona o resultado como uma nova coluna ao conjunto de dados
    df['RSI'] = rsi

    return df

In [9]:
import json
import pandas as pd
import numpy as np

RETURN_RANGE = 0.03991000000000011

# abre e carrega o conteúdo do arquivo JSON do par USDCAD
with open(f'download/usdcad-h1-bid-{from_date}-{now}.json') as f:
    data_usdcad = json.load(f)

# abre e carrega o conteúdo do arquivo JSON do Brent crude oil
with open(f'download/brentcmdusd-h1-bid-{from_date}-{now}.json') as f:
    data_oil = json.load(f)

# cria uma lista de dicionários com as informações de cada item
lista_dict = []
for item_usdcad, item_oil in zip(data_usdcad, data_oil):
    dict_item = {
        'Open_usdcad': item_usdcad['open'],
        'High_usdcad': item_usdcad['high'],
        'Low_usdcad': item_usdcad['low'],
        'Close_usdcad': item_usdcad['close'],
        'Volume_usdcad': item_usdcad['volume'],
        'return': (item_usdcad['close'] - item_usdcad['open'])/ RETURN_RANGE,
        'Open_oil': item_oil['open'],
        'High_oil': item_oil['high'],
        'Low_oil': item_oil['low'],
        'Close_oil': item_oil['close'],
        'Volume_oil': item_oil['volume'],
        'timestamp': item_usdcad['timestamp'],
    }
    lista_dict.append(dict_item)

# converte a lista de dicionários em um dataframe
df = pd.DataFrame.from_dict(lista_dict)

# aplica os indicadores técnicos ao dataframe
df = add_bollinger_bands(df)
df = add_stochastic_indicator(df)
df = add_rsi(df)


# adiciona as colunas "year", "month", "day" e "hour"
df['timestamp'] = pd.to_datetime(df['timestamp'], unit='ms')
df['year'] = df['timestamp'].apply(lambda x: x.year)
df['month'] = df['timestamp'].apply(lambda x: x.month)
df['day'] = df['timestamp'].apply(lambda x: x.day)
df['hour'] = df['timestamp'].apply(lambda x: x.hour)


df = df.drop(['timestamp', 'STD'], axis=1)
# lista com as colunas na ordem desejada
new_order = ['Open_usdcad', 'High_usdcad', 'Low_usdcad', 'Close_usdcad', 'Volume_usdcad', 'SMA', 'Upper', 'Lower', 'Stochastic', 'RSI', 'return', 'Open_oil', 'High_oil', 'Low_oil', 'Close_oil', 'Volume_oil', 'year', 'month', 'day', 'hour']

# reindexando as colunas do dataframe
df = df.reindex(columns=new_order)
# exibe o dataframe
X = df.iloc[-1]
X_2d = np.array(X).reshape(1, -1)

In [10]:
import pickle

# Carregue o modelo
with open('usdcad.pkl', 'rb') as f:
    modelo = pickle.load(f)


predicao = modelo.predict(X_2d)

print(predicao)


[0]


C:\Users\arthu\AppData\Roaming\Python\Python310\site-packages\sklearn\base.py:439: UserWarning: X does not have valid feature names, but GradientBoostingClassifier was fitted with feature names
  warnings.warn(


In [11]:
from telethon import TelegramClient
import urllib
import requests
import logging

def send_message_to_telegram_channel(text):
    messages = {
        '0': "Call H1 ⚫",
        '1': "PUT H1 🔴",
    }

    message = messages.get(text, "")

    encoded_message = urllib.parse.quote(message)
    url = "https://api.telegram.org/bot" + "6137889365:AAHlJ6w_TZ_i9QWwVsIe77HlI4X279FXlq8" + "/sendMessage?chat_id=" + "-1001816698688" + "&text=" + encoded_message

    try:
        resp = requests.get(url)
    except requests.exceptions.RequestException as e:
        logging.error("Error sending message to telegram channel: " + str(e))
        return

In [12]:
send_message_to_telegram_channel(str(predicao[0]))

In [13]:
import os

def limpar_diretorio_download():
    diretorio_download = 'download'
    for arquivo in os.listdir(diretorio_download):
        caminho_arquivo = os.path.join(diretorio_download, arquivo)
        if os.path.isfile(caminho_arquivo):
            os.remove(caminho_arquivo)


In [14]:
limpar_diretorio_download()